In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc as pm
import scipy.stats as sps
import scipy.special as spc
sns.set()

In [ ]:
### demo poisson likelihood with one data point
data = [6] 
lower,upper = 0,20

lambda_ = pm.Uniform('lambda_',lower,upper)
obs = pm.Poisson('obs',lambda_,observed=True,value=data)

model = pm.Model([lambda_,obs])
mcmc = pm.MCMC(model)
samples = mcmc.sample(50000,10000,2)
post = pd.DataFrame({'lambda' : mcmc.trace(lambda_)[:]})

In [ ]:
print (post.describe())
print (post.head())

In [ ]:
# pull lambda samples from posterior, generate simulated data based on those lambdas
nr_rows = 100000

rows = np.random.choice(post.index,replace=True,size=nr_rows)
posterior_samples = pm.rpoisson(post.iloc[rows,0])
posterior_samples

In [ ]:
### HOW DOES MCMC FIT THE LIKELIHOOD TO DATA ? 

# this attempts to solve the problem by testing a set of lambda values, and recording which of the lambdas provides 
# highest frequency of 6 (the single data point), given a number of tries per lambda.

# having one data point (6), we want to find out which value for lambda gives highest frequency of 
# matches. We do so by trying each individual value for lambda, that is, the for loop with lambda below
# acts as our (uniform) prior. 

nr_tries = 100000
out = np.zeros((upper-lower+1,len(data) * nr_tries))

r = 0
c = 0

for d in data:
    
    for lambda_ in range(lower,upper+1):
        c = 0
        for tries in range(nr_tries):
            out[r,c] = pm.rpoisson(lambda_)
            c += 1
        r += 1       

out = out.astype(int)
freq = np.count_nonzero(out == data,axis=1)
freq

In [ ]:
print (freq)
dist = freq / freq.sum()
#plt.hist(post['lambda'],density=True,alpha=0.5,color='green')
plt.bar(range(len(freq)),dist,alpha=0.5,color='red')
plt.hist(posterior_samples,density=True,alpha=0.5,color='green',bins=50)

In [ ]:
from matplotlib.ticker import MaxNLocator

ax = plt.gca()

plt.plot(np.arange(len(dist)).astype(int),dist.cumsum(),'o--')
plt.title('Cumulative Probability')
plt.xlabel('lambda')
plt.ylabel('probability')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))


In [ ]:
#### https://sciencehouse.wordpress.com/2010/06/23/mcmc-and-fitting-models-to-data/
#### https://prappleizer.github.io/Tutorials/MCMC/MCMC_Tutorial.html

#### demo of how to build a likelihood function to fit model to data using Metropolis-Hastings MCMC.
#### The problem is thus to find out what parameter value mu for the Gaussian Distribution best matches the actual data
####
# error function: sum of square errors: computes error between our data and what our model generates
# sigma is an estimate of the error of data. it seems that with a small sigma, more proposals are rejected
# but with too small sigma, runtime errors occur
#
#
#
# It appears that sigma is sensitive to both the nr of data points, the EV. 
# Finding correct value for Sigma is currently not clear to me. Most likely, Sigma must be made dynamic, 
# depending on feedback on magnitude of data, number of data points and how much the walk has converged,
# that is, after convergence has begun, Sigma should probably become smaller, to generate larger error values, 
# i.e. to obtain more fine grained errors...

def X2(data,generated,sigma=0.3) :
    return  ( ( (data - generated) ** 2 ) / (2 * sigma ** 2) ).sum() 

#### alternative error function
def X2_alt(data,generated,sigma=0.3) :
    return   ( ( (data - generated) / (data * sigma ) )  ** 2 ).sum() / 2

In [ ]:
# the data we want to fit. Here we KNOW the value for the parameter p we are looking for
# it appears that the more data we have, the clearer difference about the peak there is between PYMC
# and the hack - might be due to the error function not operating optimally

SIZE = 5 # number of data points
true_mu = 5

data = pm.rnormal(mu=true_mu,tau=1 / 1 ** 2,size=SIZE)


In [ ]:
# this is our model that we want to fit, i.e the likelihood function with parameter param for p

def generator(param,size=SIZE):
    return pm.rnormal(mu=param,tau=1 / 1 ** 2,size=size)

In [ ]:
# the function used by Metropolis-Hastings to determine wether to accept a proposal.
# it uses the value returned for current and proposed errors,X2_current,X2_proposed, to determine 
# whether to move to the proposed new value for param p

# it's basically an exponential form of the quotient proposed/current, that is, P(D|proposed_p / P(D|current_p) == 
# exp(-X2_proposed + X2_current) which is based on the Gaussian Likelihood function P(D|param) = exp(-X**2)

# The likelihood function returns:
# 1 if X2_current == X2_proposed ;
# > 1 if X2_proposed  < X2_current
# < 1 if X2_proposed > X2_current

# The returned value is then tested against a random value [0..1] to decide acceptance:
# accept = rnd < likelihood_ratio ? True : False

def likelihood_ratio(X2_current,X2_proposed):
    return np.exp(-X2_proposed + X2_current)



In [ ]:
### Metropolis-Hastings MCMC algorithm for fitting data. That is, we are using MCMC to search for 
### the parameter mu that best matches our data


steps = 100000 # length of MCMC random sampling walk

burn = steps // 2

walk = np.zeros(steps) # array of samples 

all_proposed = np.zeros(steps)
all_current = np.zeros(steps)

accepted = 0
rejected = 0

A_s = np.zeros(steps)
P_s = np.zeros(steps)

walk[0] = 1 #init first step with dummy value for param to get MCMC walk started. Good value gives better convergence

error_sigma = 0.3

# the random walk
for i in range(1,steps):
    
    current = walk[i-1]
    all_current[i] = current
    
    random_step = pm.rnormal(0, 1 / 1 ** 2)
    proposed = current + random_step
        
    all_proposed[i] = proposed
    
    error_func = X2
    
    X2_current = error_func(data,generator(current),error_sigma) #compute error of current generated data vs real data
    X2_proposed = error_func(data,generator(proposed),error_sigma) #compute error of proposed generated data vs real data
    
    A = likelihood_ratio(X2_current,X2_proposed) # compute ratio, i.e accept ? 
    A_s[i-1] = A
    
    # ratio above expresses ratio of probabilities for proposed outcome vs current outcome, accoriding to distribution
    # if ratio > 1 : accept always. if ratio < 1, accept if ratio > random number 0..1
    # That is: if P(target) > P(current) : always accept. Else accept if random p is less than ratio. The smaller
    # the ratio, the less chance of accept. 
    
    p = pm.runiform(0,1)
    P_s[i-1] = p
    

    if p < A : 
        walk[i] = proposed # accept
        accepted += 1
    else:
        walk[i] = current
        rejected += 1
        
    
        
print ('accepted',accepted,'rejected',rejected,'accepted/(accepted + rejected)', accepted / (accepted + rejected))
 


In [ ]:
if steps < 1e6:
    alpha =  1 / np.log10(steps) / np.log10(steps)

    plt.figure(figsize=(18,12))
    plt.title('Likelihood Ratio vs Random draw - if Random Draw less than LR, accept proposed')
    plt.plot(range(1,len(A_s) ),A_s[:-1],'o--',color='blue',label='Likelihood Ratio',alpha=alpha)
    plt.plot(range(1,len(P_s)),P_s[:-1],'o--',color='orange',label='Random draw',alpha=alpha)
    plt.xlabel('to position')
    plt.legend(loc='upper left')
    plt.yscale('log')

In [ ]:
if steps < 1e6:
    plt.figure(figsize=(18,12))
    plt.title('Metropolis-Hastings random sampling walk,True parameter value:{} nr of data points: {}'.format(true_mu,len(data)))
    plt.xlabel ('step number')
    plt.ylabel(' acceepted and proposed parameter values')
    plt.plot(walk,'o--',label='accepted',color='navy',alpha=alpha)
    plt.plot(range(1,len(all_proposed)),all_proposed[1:],'o',color='orange',label='proposed',alpha=alpha)
    plt.legend(loc='upper left')

In [ ]:
pdf = sps.norm.pdf(np.linspace(0,9,100),true_mu,1)
counts,bins = np.histogram(walk[:burn],bins=[0,1,2,3,4,5,6,7,8,9,10])
print (counts,bins)

print (walk[burn:].mean())
print (walk[burn:].std())

In [ ]:
plt.figure(figsize=(18,12))
plt.title('MCMC Metropolis Hastings with Gaussian Likelihood'\
          ' True Parameter value: {}\n nr of data points: {} nr of steps: {}' .format(true_mu,len(data),steps))
plt.xlabel('parameter value')
plt.ylabel('Relative Frequency')

plt.plot(np.linspace(0,9,100),pdf,color='crimson',
         ls='dashed',label=r'Normal PDF $\mu$: {} $\sigma$: {}'.format(true_mu,1))

_=plt.hist(walk[burn:],color='orange',
           weights=np.ones_like(walk[burn:]) / len(walk[burn:]),label='MCMC Hack')
plt.legend(loc='upper left')
plt.savefig('MCMC_HACK.jpg',format='jpg')

In [ ]:
# use PYMC for the same fitting, using uniform prior

#prior = pm.Uniform('prior',0,1)
#obs = pm.Binomial('obs',n=N,p=prior,observed=True,value=data)

prior = pm.Uniform('prior',-10,10)
obs = pm.Normal('obs',mu=prior,tau= 1 / 1 ** 2,observed=True,value=data)
                  
model = pm.Model([prior,obs])
map_ = pm.MAP(model)
map_.fit()
mcmc = pm.MCMC(model)
samples = mcmc.sample(50000,10000,2)

result = pd.DataFrame({'post_prior' :mcmc.trace(prior)[:]})


In [ ]:
result.describe()

In [ ]:
ci_89 = np.percentile(walk[burn:],[5.5,94.5])

plt.figure(figsize=(18,12))
plt.hist(result.post_prior,density=True,label='PYMC',alpha=0.6,color='blue',bins=10)
plt.hist(walk[burn:],density=True,label='MCMC-hack',alpha=0.6,color='orange',bins=20)
plt.axvline(ci_89[0],color='red',ls='dashed')
plt.axvline(ci_89[1],color='red',ls='dashed')
plt.plot(np.linspace(0,9,100),pdf,color='orange',ls='dashed',label=r'Normal PDF $\mu$: {} $\sigma$:{}'.format(true_mu,1))
plt.legend(loc='upper left')

In [ ]:
### a look at the basics of a Gaussian
x = np.linspace(-3,3,100)
y = np.exp(-x**2)
plt.plot(x,y)


In [ ]:
def lkh_ratio(X2_current,X2_proposed):
    return np.exp(-X2_proposed + X2_current) 


def X2_alt(data,generated,sigma=0.3) :
    return  (  ( (data - generated) / data )  ** 2 ).sum() / 2

D = np.array([5,5,5,5,5] )

curr = np.array([4,4,4,4,4] ) 
prop = np.array([4,4,4,4,5] ) 

sigma = 1

e_curr = X2(D,curr,sigma)
e_prop = X2(D,prop,sigma)

print ('e_curr ',e_curr)
print ('e_prop ',e_prop)
ratio = lkh_ratio(e_curr,e_prop)
print ('ratio ',ratio)

p = pm.runiform(0,1)
print ('p ',p)
accpt = p < ratio
print ('accept ? ',accpt)

In [ ]:
# ratios = np.exp(-e_prop) / np.exp(-e_curr) can be done without division as below

ratios = np.exp(-e_prop + e_curr)
print (ratios)
print (ratios2)

In [ ]:


for e_prop in range (4):
    label='e_prop: {}'.format(e_prop)
    ratios = []
    print ()
    for e_curr in range(4):
        print (e_curr,e_prop,lkh_ratio(e_curr,e_prop))
        ratios.append(lkh_ratio(e_curr,e_prop))
    plt.plot(ratios,'o--',label=label)

plt.xlabel('e_curr')
plt.ylabel('likelihood ratio')
plt.legend(loc='upper left')

